In [1]:
import zipfile
import base64
import io
import glob
import time
import json
import os
import requests
import mimetypes
import pandas as pd
from bs4 import BeautifulSoup
import re
from urllib.parse import urlencode
from urllib.parse import quote_plus
import tika
from tika import parser
pd.set_option("max_colwidth",200)

/Users/hiromi/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Use 'bigtech_lobby - formatted.csv' 
This csv is already reflected amendament & filter 2010-2021. I did this work on Google spreadsheet. 

In [2]:
df_bigtech = pd.read_csv('bigtech_lobby - formatted.csv')

In [3]:
df_bigtech.head(3)

,season,year,amazon_lobby,facebook_lobby,google_lobby,apple_lobby,microsoft_lobby,amazon_link,facebook_link,google_link,apple_link,microsoft_link
0,2021 2nd Quarter - Report,2021,4860000,4770000,2090000,1640000,2470000,https://lda.senate.gov//filings/public/filing/532894c6-bc65-4e48-b7e9-8c2747fa5426/print/,https://lda.senate.gov//filings/public/filing/20bc0f7b-a3d0-4c00-bc5d-72a405ff16e7/print/,https://lda.senate.gov//filings/public/filing/bf977a89-15c2-41f5-b59d-7e7a6e17b210/print/,https://lda.senate.gov//filings/public/filing/008a3286-d6d5-4601-afe3-f4c4361785fc/print/,https://lda.senate.gov//filings/public/filing/fad755f6-ba3a-480c-a3a1-678bb14c0a5a/print/
1,2021 1st Quarter - Report,2021,4800000,4790000,2690000,1460000,2590000,https://lda.senate.gov//filings/public/filing/a87a3a40-7aa3-4b09-a90a-1ae233f3f69f/print/,https://lda.senate.gov//filings/public/filing/75504307-e84f-4819-9573-25c28867b498/print/,https://lda.senate.gov//filings/public/filing/047f925e-aa0c-46fa-95aa-f78c29b4f5ee/print/,https://lda.senate.gov//filings/public/filing/931b23da-8021-4f33-8f4b-70e571dba9b8/print/,https://lda.senate.gov//filings/public/filing/d50d3ddf-86ca-40e6-8f35-91d66edc8d92/print/
2,2020 4th Quarter - Report,2020,4740000,4690000,2110000,1450000,2190000,https://lda.senate.gov//filings/public/filing/57ffc505-528c-4de6-a5fc-9c3c9a01b798/print/,https://lda.senate.gov//filings/public/filing/9caddbc5-946a-4148-8ef3-acc0a97e721f/print/,https://lda.senate.gov//filings/public/filing/65749c45-3697-492e-8430-520cfa2fd6ad/print/,https://lda.senate.gov//filings/public/filing/cad6db2f-c3ca-4b9d-bc24-4c56fc7eaadb/print/,https://lda.senate.gov//filings/public/filing/0906e6e8-20f7-4e95-a649-68381ad16c87/print/


In [4]:
df_bigtech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   season           46 non-null     object
 1   year             46 non-null     int64 
 2   amazon_lobby     46 non-null     int64 
 3   facebook_lobby   46 non-null     int64 
 4   google_lobby     46 non-null     int64 
 5   apple_lobby      46 non-null     int64 
 6   microsoft_lobby  46 non-null     int64 
 7   amazon_link      46 non-null     object
 8   facebook_link    46 non-null     object
 9   google_link      46 non-null     object
 10  apple_link       46 non-null     object
 11  microsoft_link   46 non-null     object
dtypes: int64(6), object(6)
memory usage: 4.4+ KB


In [ ]:
# crt_bill_text = []

# for bill in crt_states_df['bill_url']:
#     parsed = parser.from_file(bill)
#     bill_text = parsed['content'].strip()
#     crt_bill_text.append(bill_text)
    
# crt_states_df['bill_text'] = crt_bill_text

In [5]:
amazon_text = []

for pdf in df_bigtech['amazon_link']:
    try:
        parsed = parser.from_file(pdf)
        text = parsed['content'].strip().replace('\n',' ').replace('\t', ' ').replace(u'\u200b', ' ') 
    except:
        text = 'no PDF'
    amazon_text.append(text)

2021-08-07 15:36:27,752 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/532894c6-bc65-4e48-b7e9-8c2747fa5426/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-532894c6-bc65-4e48-b7e9-8c2747fa5426-print.
2021-08-07 15:36:28,393 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/a87a3a40-7aa3-4b09-a90a-1ae233f3f69f/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-a87a3a40-7aa3-4b09-a90a-1ae233f3f69f-print.
2021-08-07 15:36:28,695 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/57ffc505-528c-4de6-a5fc-9c3c9a01b798/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-57ffc505-528c-4de6-a5fc-9c3c9a01b798-print.
2021-08-07 15:36:29,000 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/1c54ad52-6bd7-4474-aeb4-c2ebaaa907c7/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

2021-08-07 15:36:37,377 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/72477a97-5625-45ad-9dfa-c3461bb4e5ea/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-72477a97-5625-45ad-9dfa-c3461bb4e5ea-print.
2021-08-07 15:36:37,633 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/26fea122-15fc-41c9-941f-478036ed8e4d/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-26fea122-15fc-41c9-941f-478036ed8e4d-print.
2021-08-07 15:36:37,921 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/87c1525f-8af5-4b18-9a1b-ab4c6b5a4c33/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-87c1525f-8af5-4b18-9a1b-ab4c6b5a4c33-print.
2021-08-07 15:36:38,161 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/5445c45e-4666-443a-b06a-76783626c859/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

In [6]:
df_bigtech['amazon_text'] = amazon_text

In [7]:
facebook_text = []

for pdf in df_bigtech['facebook_link']:
    try:
        parsed = parser.from_file(pdf)
        text = parsed['content'].strip().replace('\n',' ').replace('\t', ' ').replace(u'\u200b', ' ') 
    except:
        text = 'no PDF'
    facebook_text.append(text)

2021-08-07 15:37:53,263 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/20bc0f7b-a3d0-4c00-bc5d-72a405ff16e7/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-20bc0f7b-a3d0-4c00-bc5d-72a405ff16e7-print.
2021-08-07 15:37:53,837 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/75504307-e84f-4819-9573-25c28867b498/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-75504307-e84f-4819-9573-25c28867b498-print.
2021-08-07 15:37:59,444 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/9caddbc5-946a-4148-8ef3-acc0a97e721f/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-9caddbc5-946a-4148-8ef3-acc0a97e721f-print.
2021-08-07 15:38:00,504 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/ddb2d021-2469-4697-a44c-c3aaf959f7d8/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

2021-08-07 15:38:28,401 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/19b20cd2-4c2f-4f41-ab05-8622779a918a/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-19b20cd2-4c2f-4f41-ab05-8622779a918a-print.
2021-08-07 15:38:28,643 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/9f11cfa2-ea4b-422a-a026-51b0f14e883b/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-9f11cfa2-ea4b-422a-a026-51b0f14e883b-print.
2021-08-07 15:38:29,089 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/bee30eb0-bf20-47cd-b00d-f5f2a9d10aa6/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-bee30eb0-bf20-47cd-b00d-f5f2a9d10aa6-print.
2021-08-07 15:38:29,370 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/3e77cac4-4b25-4ede-84d7-80304f7cb24d/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

In [8]:
df_bigtech['facebook_text'] = facebook_text

In [9]:
google_text = []

for pdf in df_bigtech['google_link']:
    try:
        parsed = parser.from_file(pdf)
        text = parsed['content'].strip().replace('\n',' ').replace('\t', ' ').replace(u'\u200b', ' ') 
    except:
        text = 'no PDF'
    google_text.append(text)

2021-08-07 15:38:43,902 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/bf977a89-15c2-41f5-b59d-7e7a6e17b210/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-bf977a89-15c2-41f5-b59d-7e7a6e17b210-print.
2021-08-07 15:38:44,199 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/047f925e-aa0c-46fa-95aa-f78c29b4f5ee/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-047f925e-aa0c-46fa-95aa-f78c29b4f5ee-print.
2021-08-07 15:38:44,808 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/65749c45-3697-492e-8430-520cfa2fd6ad/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-65749c45-3697-492e-8430-520cfa2fd6ad-print.
2021-08-07 15:38:45,116 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/40df1c3a-c0d4-42cd-898a-e4ff6dac3eab/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

2021-08-07 15:38:55,211 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/a36c2662-c1c5-47c9-816b-718c90ebb300/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-a36c2662-c1c5-47c9-816b-718c90ebb300-print.
2021-08-07 15:38:55,621 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/999c7d0c-8eb1-44c9-993f-a3cf969d90e8/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-999c7d0c-8eb1-44c9-993f-a3cf969d90e8-print.
2021-08-07 15:38:55,912 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/4cb55b30-1f79-4ea9-961b-985b73b6917e/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-4cb55b30-1f79-4ea9-961b-985b73b6917e-print.
2021-08-07 15:38:56,222 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/5e1012c0-6d42-4bb7-b363-f8649ba4edda/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

In [10]:
df_bigtech['google_text'] = google_text

In [11]:
apple_text = []

for pdf in df_bigtech['apple_link']:
    try:
        parsed = parser.from_file(pdf)
        text = parsed['content'].strip().replace('\n',' ').replace('\t', ' ').replace(u'\u200b', ' ') 
    except:
        text = 'no PDF'
    apple_text.append(text)

2021-08-07 15:39:30,601 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/008a3286-d6d5-4601-afe3-f4c4361785fc/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-008a3286-d6d5-4601-afe3-f4c4361785fc-print.
2021-08-07 15:39:31,183 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/931b23da-8021-4f33-8f4b-70e571dba9b8/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-931b23da-8021-4f33-8f4b-70e571dba9b8-print.
2021-08-07 15:39:38,282 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/cad6db2f-c3ca-4b9d-bc24-4c56fc7eaadb/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-cad6db2f-c3ca-4b9d-bc24-4c56fc7eaadb-print.
2021-08-07 15:39:38,604 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/ca972322-9352-425e-be3f-ba820f6fc1f7/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

2021-08-07 15:40:09,504 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/0196269c-0c21-49ea-aa65-ee078bfa1f1b/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-0196269c-0c21-49ea-aa65-ee078bfa1f1b-print.
2021-08-07 15:40:10,404 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/c9c15a6f-182f-492b-a104-71c2df94d786/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-c9c15a6f-182f-492b-a104-71c2df94d786-print.
2021-08-07 15:40:11,207 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/242ad532-4064-4c1c-a408-5eb65c25bb9e/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-242ad532-4064-4c1c-a408-5eb65c25bb9e-print.
2021-08-07 15:40:24,941 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/2a14a8ba-3b99-4bf4-a45d-31a341c9f092/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

In [12]:
df_bigtech['apple_text'] = apple_text

In [13]:
microsoft_text = []

for pdf in df_bigtech['microsoft_link']:
    try:
        parsed = parser.from_file(pdf)
        text = parsed['content'].strip().replace('\n',' ').replace('\t', ' ').replace(u'\u200b', ' ') 
    except:
        text = 'no PDF'
    microsoft_text.append(text)

2021-08-07 15:41:21,332 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/fad755f6-ba3a-480c-a3a1-678bb14c0a5a/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-fad755f6-ba3a-480c-a3a1-678bb14c0a5a-print.
2021-08-07 15:41:21,716 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/d50d3ddf-86ca-40e6-8f35-91d66edc8d92/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-d50d3ddf-86ca-40e6-8f35-91d66edc8d92-print.
2021-08-07 15:41:22,961 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/0906e6e8-20f7-4e95-a649-68381ad16c87/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-0906e6e8-20f7-4e95-a649-68381ad16c87-print.
2021-08-07 15:41:24,135 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/11832112-0bf4-489a-a552-b64a87401620/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

2021-08-07 15:43:01,079 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/e106dded-6546-4a9e-b1df-bf282b434f3b/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-e106dded-6546-4a9e-b1df-bf282b434f3b-print.
2021-08-07 15:43:01,366 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/f43f5de8-0f33-4103-aef1-afb81d8f2255/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-f43f5de8-0f33-4103-aef1-afb81d8f2255-print.
2021-08-07 15:43:02,803 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/c696210e-819c-412a-98ae-78d4b9fd187b/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/T/filings-public-filing-c696210e-819c-412a-98ae-78d4b9fd187b-print.
2021-08-07 15:43:03,678 [MainThread  ] [INFO ]  Retrieving https://lda.senate.gov//filings/public/filing/6e13e7d9-8d00-460b-81f1-a4ec8821c531/print/ to /var/folders/l4/syqwd3ts14b0swly1g_hr3h40000gn/

In [14]:
df_bigtech['microsoft_text'] = microsoft_text

In [18]:
df_bigtech.head(3)

,season,year,amazon_lobby,facebook_lobby,google_lobby,apple_lobby,microsoft_lobby,amazon_link,facebook_link,google_link,apple_link,microsoft_link,amazon_text,facebook_text,google_text,apple_text,microsoft_text
0,2021 2nd Quarter - Report,2021,4860000,4770000,2090000,1640000,2470000,https://lda.senate.gov//filings/public/filing/532894c6-bc65-4e48-b7e9-8c2747fa5426/print/,https://lda.senate.gov//filings/public/filing/20bc0f7b-a3d0-4c00-bc5d-72a405ff16e7/print/,https://lda.senate.gov//filings/public/filing/bf977a89-15c2-41f5-b59d-7e7a6e17b210/print/,https://lda.senate.gov//filings/public/filing/008a3286-d6d5-4601-afe3-f4c4361785fc/print/,https://lda.senate.gov//filings/public/filing/fad755f6-ba3a-480c-a3a1-678bb14c0a5a/print/,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...
1,2021 1st Quarter - Report,2021,4800000,4790000,2690000,1460000,2590000,https://lda.senate.gov//filings/public/filing/a87a3a40-7aa3-4b09-a90a-1ae233f3f69f/print/,https://lda.senate.gov//filings/public/filing/75504307-e84f-4819-9573-25c28867b498/print/,https://lda.senate.gov//filings/public/filing/047f925e-aa0c-46fa-95aa-f78c29b4f5ee/print/,https://lda.senate.gov//filings/public/filing/931b23da-8021-4f33-8f4b-70e571dba9b8/print/,https://lda.senate.gov//filings/public/filing/d50d3ddf-86ca-40e6-8f35-91d66edc8d92/print/,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...
2,2020 4th Quarter - Report,2020,4740000,4690000,2110000,1450000,2190000,https://lda.senate.gov//filings/public/filing/57ffc505-528c-4de6-a5fc-9c3c9a01b798/print/,https://lda.senate.gov//filings/public/filing/9caddbc5-946a-4148-8ef3-acc0a97e721f/print/,https://lda.senate.gov//filings/public/filing/65749c45-3697-492e-8430-520cfa2fd6ad/print/,https://lda.senate.gov//filings/public/filing/cad6db2f-c3ca-4b9d-bc24-4c56fc7eaadb/print/,https://lda.senate.gov//filings/public/filing/0906e6e8-20f7-4e95-a649-68381ad16c87/print/,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...,LD-2 Disclosure Form Clerk of the House of Representatives Legislative Resource Center 135 Cannon Building ...


In [1]:
# df_bigtech.info()

## Create Big tech csv for project

In [19]:
df_bigtech.to_csv('bigtecg_lobby_detail_2010-2021.csv', index=False)

## Generate company's CSVs if needed

In [44]:
# df_amazon = df_bigtech[['season','year','amazon_lobby','amazon_link', 'amazon_text']]
# df_amazon.to_csv('amazon_lobby.csv', index=False)

In [45]:
# df_facebook = df_bigtech[['season','year','facebook_lobby','facebook_link', 'facebook_text']]
# df_facebook.to_csv('facebook_lobby.csv', index=False)

In [46]:
# df_google = df_bigtech[['season','year','google_lobby','google_link', 'google_text']]
# df_google.to_csv('google_lobby.csv',index=False)

In [49]:
# df_apple = df_bigtech[['season','year','apple_lobby','apple_link', 'apple_text']]
# df_apple.to_csv('apple_lobby.csv',index=False)

In [52]:
# df_microsoft = df_bigtech[['season','year','microsoft_lobby','microsoft_link', 'microsoft_text']]
# df_apple.to_csv('microsoft_lobby.csv',index=False)